In [141]:
from google.cloud import bigquery

In [142]:
import pandas as pd

In [143]:
class GoogleAnalyticsSample:
    """
    Class to pull the data from the Google Analytics Sample.
    
    Short information about this dataset:
    
    The dataset provides 12 months (August 2016 to August 2017) 
    of obfuscated Google Analytics 360 data from 
    the Google Merchandise Store 
    
    Data for some fields is obfuscated such as fullVisitorId, 
    or removed such as clientId, adWordsClickInfo and geoNetwork. 
    “Not available in demo dataset” will be returned for STRING values and 
    “null” will be returned for INTEGER values when querying the fields containing no data.
    
    Dataset has 5.4 GB.
    
    """
    
    def __init__(self):
        """
        
        Init function contructing the client and authenticating 
        with credentials set in the environment
        and performing the SQL query leaving out the obfuscated or removed columns from the dataset
        
        """

        self.client = bigquery.Client()

        self.query = """
        SELECT
              date, sum(totals.hits), sum(totals.pageviews), trafficSource.campaign, trafficSource.source,
              device.browser, device.operatingSystem, device.isMobile, device.deviceCategory,
              geoNetwork.continent, geoNetwork.subContinent, geoNetwork.country,
              channelGrouping,
              socialEngagementType
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_20161210`
    
        GROUP BY 
            date, trafficSource.campaign, trafficSource.source, device.browser, device.operatingSystem,
            device.isMobile, device.deviceCategory, geoNetwork.continent, geoNetwork.subContinent, geoNetwork.country,
            channelGrouping, socialEngagementType
    
         """
        self.query_job = self.client.query(self.query)
        self.results = self.query_job.result()
        
        
    def print_first_x_results(self, x):
        """
        Function to print the first x rows from the results of the query
        
        x: int, number of results that will be printed
        """
        if type(x) not in (int, float):
            raise TypeError('X should be an integer number')

        i = 0
        for row in self.results :
            print(row)
            i +=1
            if i == x:
                break
                    
    
    def to_dataframe(self):
        """
        Outputs the results as a Pandas dataframe
        """
        
        return self.query_job.to_dataframe()
    
    
    def how_many_countries(self):
        """
        Outputs how many unique countries are coming up in the dataset
        using the unique() method in pandas
        """
        
        df = self.to_dataframe()
        
        return len(pd.unique(df['country']))
    
    
    def which_browsers(self):
        """
        Output the a list of browsers that come up in the dataset
        using the unique() method in pandas
        """
        
        df_browser = self.to_dataframe()
    
        return pd.unique(df['browser'])
    
    def print_results(self, result):
        """
        Function can be used to print any result of any SQL query 
        
        result: rowIterator object resulting from the big query method result()
        """
        
        for row in result:
            print(row)
    
    def how_many_pageviews(self):
        
        """
        Function outputs the total sum of pageviews in the Google Merchandise Store
        the result is send to the print_results method of this class
        """
        
        self.query_pageviews = """
        SELECT sum(totals.pageviews)
        FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_20161210`
        """
    
        self.query_job_pageviews = self.client.query(self.query_pageviews)
        self.results_pageviews = self.query_job_pageviews.result()
        
        return self.print_results(self.results_pageviews)
    
    

In [144]:
googleanalytics = GoogleAnalyticsSample()

In [145]:
googleanalytics.print_first_x_results(5)

Row(('20161210', 1, 1, '(not set)', '(direct)', 'Chrome', 'Macintosh', False, 'desktop', 'Europe', 'Southern Europe', 'Italy', 'Organic Search', 'Not Socially Engaged'), {'date': 0, 'f0_': 1, 'f1_': 2, 'campaign': 3, 'source': 4, 'browser': 5, 'operatingSystem': 6, 'isMobile': 7, 'deviceCategory': 8, 'continent': 9, 'subContinent': 10, 'country': 11, 'channelGrouping': 12, 'socialEngagementType': 13})
Row(('20161210', 7, 7, '(not set)', '(direct)', 'Chrome', 'Windows', False, 'desktop', 'Asia', 'Eastern Asia', 'South Korea', 'Organic Search', 'Not Socially Engaged'), {'date': 0, 'f0_': 1, 'f1_': 2, 'campaign': 3, 'source': 4, 'browser': 5, 'operatingSystem': 6, 'isMobile': 7, 'deviceCategory': 8, 'continent': 9, 'subContinent': 10, 'country': 11, 'channelGrouping': 12, 'socialEngagementType': 13})
Row(('20161210', 224, 190, '(not set)', '(direct)', 'Chrome', 'Macintosh', False, 'desktop', 'Americas', 'Northern America', 'United States', 'Paid Search', 'Not Socially Engaged'), {'date': 

In [146]:
df = googleanalytics.to_dataframe()

In [147]:
df

,date,f0_,f1_,campaign,source,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,channelGrouping,socialEngagementType
0,20161210,1,1,(not set),(direct),Chrome,Macintosh,False,desktop,Europe,Southern Europe,Italy,Organic Search,Not Socially Engaged
1,20161210,7,7,(not set),(direct),Chrome,Windows,False,desktop,Asia,Eastern Asia,South Korea,Organic Search,Not Socially Engaged
2,20161210,224,190,(not set),(direct),Chrome,Macintosh,False,desktop,Americas,Northern America,United States,Paid Search,Not Socially Engaged
3,20161210,2,2,(not set),(direct),Chrome,Windows,False,desktop,Asia,Southeast Asia,Thailand,Organic Search,Not Socially Engaged
4,20161210,599,526,(not set),(direct),Safari,iOS,True,mobile,Americas,Northern America,United States,Organic Search,Not Socially Engaged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,20161210,6,6,(not set),reddit.com,Chrome,iOS,True,tablet,Asia,Western Asia,Israel,Social,Not Socially Engaged
671,20161210,8,6,(not set),youtube.com,Firefox,Linux,False,desktop,Americas,South America,Venezuela,Social,Not Socially Engaged
672,20161210,11,8,(not set),youtube.com,Chrome,Android,True,tablet,Americas,Northern America,United States,Social,Not Socially Engaged
673,20161210,13,13,(not set),reddit.com,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,Social,Not Socially Engaged


In [148]:
googleanalytics.how_many_countries()

100

In [149]:
googleanalytics.which_browsers()

array(['Chrome', 'Safari', 'Edge', 'Opera Mini', 'Firefox',
       'Internet Explorer', 'BlackBerry', 'MQQBrowser', 'Amazon Silk',
       'UC Browser', 'Opera', 'Safari (in-app)', 'Coc Coc', 'YaBrowser',
       'Nintendo Browser', 'MRCHROME', 'Android Browser'], dtype=object)

In [150]:
googleanalytics.how_many_pageviews()

Row((9693,), {'f0_': 0})
